In [2]:
import pandas as pd
import json
from pandas import Series,DataFrame
import re
import datetime
pd.set_option('display.max_rows',20)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',20)

item=[]
for line in open('lianjia.json','r',encoding='utf-8'):
    item.append(json.loads(line))
    
df=DataFrame(item)

In [3]:
for i in range(0,len(df)):
    result=df.iloc[i]
    if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
        print(df.iloc[i])
        df.drop([i])

In [4]:
getarea=lambda x: float(re.search(r"\d+(\.\d+)?",x).group())
df['area']=df['area'].apply(getarea)
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))


In [5]:
df.head

<bound method NDFrame.head of        title  room    area   dealDate  totalPrice  unitPrice  dealCycle
0       城市印象  2室1厅   84.50 2019-03-26       119.0    14024.0        227
1       城市印象  2室2厅   91.62 2019-03-20       174.0    18992.0        242
2       城市印象  2室1厅   85.34 2019-03-09       107.0    12480.0        120
3       城市印象  2室2厅   84.50 2019-03-05       115.0    13610.0        119
4       城市印象  2室2厅   88.51 2019-02-27       189.0    21354.0        235
...      ...   ...     ...        ...         ...        ...        ...
26298  新希望家园  2室1厅   56.07 2020-02-26        45.0     7862.0        268
26299  新希望家园  1室1厅   33.73 2020-02-22        28.0     8302.0        455
26300  新希望家园  3室1厅  103.00 2020-02-21        97.0     9350.0         36
26301  新希望家园  2室1厅   78.00 2020-01-11        68.0     8718.0        138
26302  新希望家园  2室1厅   76.03 2020-01-08        77.0    10128.0         88

[26303 rows x 7 columns]>

In [12]:
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]

In [36]:
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)

In [37]:
df_wu

,title,room,area,dealDate,totalPrice,unitPrice,dealCycle
0,第五郡五号地,2室2厅,90.28,2016-03-23,158.0,17502.0,37
1,第五郡五号地,3室2厅,122.49,2016-05-05,224.0,18247.0,62
2,第五郡五号地,3室2厅,136.00,2016-07-17,230.0,16912.0,116
3,第五郡五号地,4室3厅,125.04,2016-07-22,229.0,18315.0,138
4,第五郡五号地,2室2厅,89.92,2016-08-05,150.0,16682.0,75
...,...,...,...,...,...,...,...
86,第五郡五号地,2室1厅,86.10,2021-04-19,218.0,25320.0,203
87,第五郡五号地,4室2厅,170.62,2021-04-23,473.0,27723.0,308
88,第五郡五号地,4室2厅,130.98,2021-04-24,372.0,28402.0,66
89,第五郡五号地,3室1厅,136.00,2021-04-30,381.0,27978.0,105


In [40]:
df['title'].drop_duplicates()

0             城市印象
27             香炉礁
41          龙畔金泉五期
55       龙畔金泉五期K4区
130         万科假日风景
           ...    
26000         朗悦华园
26040       福佳机场前院
26048       亿达普罗旺斯
26085          东港第
26115         东城国际
Name: title, Length: 259, dtype: object